# What's that part of the graph look like?

This notebook is intended purely for visualizing parts of the graph. It also includes functions that allow us to color the graph nodes to show different variables such as track duration (default), or estimated number of worms.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import print_function, absolute_import, unicode_literals, division
import six
from six.moves import (zip, filter, map, reduce, input, range)

import pathcustomize

import sys
import os
import pathlib
import pickle
import platform
print('Python {} ({}) [{}] on {}'.format(platform.python_version(), ', '.join(platform.python_build()), 
                                       platform.python_compiler(), sys.platform))

import matplotlib.pyplot as plt
import matplotlib.colors as mplcolors
import pandas as pd
import networkx as nx
from IPython.core.display import Image as display_image 

import waldo
import waldo.viz as viz

Python 2.7.9 (default, Dec 15 2014 10:37:34) [GCC 4.2.1 (Apple Inc. build 5577)] on darwin


In [3]:
#ex_id = '20130614_120518'
#ex_id = '20130318_131111'
#ex_id = '20130414_140704'
ex_id = '20130702_135704' # many pics
#ex_id = '20130702_135652' # many pics

experiment = waldo.Experiment(experiment_id=ex_id)
graph = experiment.graph.copy()

waldo.collider.remove_nodes_outside_roi(graph, experiment)
waldo.collider.remove_blank_nodes(graph, experiment)

print('N = {}'.format(len(graph)))
#collider.component_size_summary(graph)
viz.direct_degree_distribution(graph, (3, 3), ignore_zero=False)

MWTSummaryError: Could not find summary file in target path: /Users/peterwinter/Data/waldo/20130702_135704

# Create simplified giant component

In [ ]:
slightly_simplified_graph = graph.copy()

waldo.collider.remove_single_descendents(slightly_simplified_graph)
waldo.collider.remove_fission_fusion(slightly_simplified_graph)
waldo.collider.remove_fission_fusion_rel(slightly_simplified_graph, split_rel_time=0.5)
waldo.collider.remove_offshoots(slightly_simplified_graph, threshold=20)
waldo.collider.remove_single_descendents(slightly_simplified_graph)
viz.direct_degree_distribution(slightly_simplified_graph, (3, 3), ignore_zero=False)

In [ ]:
giant = slightly_simplified_graph.giant()
nodes = giant.nodes(data=False)
print(len(nodes), 'in giant component')
target = nodes[0]
print(target)

In [ ]:
cmap = 'jet'
#cmap = 'Paired'
cmap = 'bwr'
cmap = 'afmhot'
cmap = 'pink'

cm = plt.cm.get_cmap(cmap)
def lifespan_to_color(lifespan, scalerange=(-0.5, 4), cm=cm):
    val = (np.log10(lifespan+1e-100) - scalerange[0])/(scalerange[1] - scalerange[0])                                            
    color = mplcolors.rgb2hex(cm(val)[:3]) 
    return color     
    
ref_graph= nx.DiGraph()
reference = [0, 1, 3, 10, 30, 100, 300, 1000, 3000]
ref_nodes = ['{} frames'.format(r) for r in reference]
iterref = iter(ref_nodes)
source = six.next(iterref)
for dest in iterref:
    ref_graph.add_edge(source, dest)
    source = dest
    for ref, ref_node in zip(reference, ref_nodes):
        ref_graph.add_node(ref_node, penwidth='0.5',
                         fillcolor=lifespan_to_color(ref), style='filled')
temp_file = viz.network.dot.render_nx_as_dot(ref_graph, remove_node_labels=True, output_file='life-ref.svg', format='png')
display_image(temp_file)

In [ ]:
subgraph = viz.subgraph.nearby(slightly_simplified_graph, target, 1000) 
viz.network.dot.format_graph_for_lifespan(subgraph, cmap=cmap, ref=False)
temp_file = viz.network.dot.render_nx_as_dot(subgraph, remove_node_labels=True, output_file='life-giant.gv', format='png')
display_image(temp_file)

In [ ]:
viz.look(slightly_simplified_graph, target=target, jumps=1000, ref=0, ctype='lifespan')

### collect data for true counts

### Show worm number estimation

In [ ]:
slightly_simplified_graph.count_worms(experiment)
viz.look(slightly_simplified_graph, target=target, jumps=1000, ref=0,ctype='true_count')

In [ ]:
target = 15074
jumps = 12

In [ ]:
target = 13057
jumps = 4

In [ ]:
#target = 491
#jumps = 8

In [ ]:
slightly_simplified_graph.determine_moving(experiment)
viz.look(slightly_simplified_graph, target=target, jumps=jumps, ref=0, ctype='moved_bool')

In [ ]:
slightly_simplified_graph.count_worms(experiment)
viz.look(slightly_simplified_graph, target=target, jumps=jumps, ref=0,ctype='worm_count')

In [ ]:
screen = pd.read_csv('../data/prep/collision_validate.csv')
screen = screen[screen['eid'] == ex_id]
assert len(screen), "No data for experiment ID"
screen.set_index('bid', inplace=True)
#screen.head()

true_counts = {}

for bid, row in screen.iterrows():
    c = int(row['ans'] / 10)
    if not c:
        c = 0.5
    #print(type(bid))
    true_counts[int(bid)] = c
    
slightly_simplified_graph.add_node_attributes('true_count', true_counts, default=0)
viz.look(slightly_simplified_graph, target=target, jumps=jumps, ref=0, ctype='true_count')

In [ ]:
viz.save_graphs(ex_id, slightly_simplified_graph, target=target, jumps=jumps)

### zooming in on individual targets

In [ ]:
graph2 = graph.copy()
params = {
    'offshoots': 20, # default: 20
    'splits_abs': 5, # default: 5
    'splits_rel': 0.25, # default: 0.5
    'assimilate': -1, # default: 10
}
waldo.collider.removal_suite(graph2, **params)
#collider.remove_offshoots(graph2, params['offshoots'])
#collider.assimilate(graph2, params['assimilate'])
removed = waldo.collider.remove_blank_nodes(graph2, experiment, exclude_connected=False)
print('N = {}'.format(len(graph2)))
waldo.collider.component_size_summary(graph2)
waldo.viz.direct_degree_distribution(graph2, (3, 3), ignore_zero=False)
print(list(removed))

In [ ]:
viz.look(experiment.graph, target=13014, jumps=10, ref=1)

In [ ]:
viz.look(experiment.graph, target=11133, jumps=10, ref=1)

In [ ]:
t = 17990
tspan = 1000
viz.look_time(graph2, fstart=t, fend=t+tspan)

In [ ]:
t = 17990
tspan = 1
look_time(experiment.graph, fstart=t, fend=t+tspan)

In [ ]:
t = 17990
tspan = 1
look_time(graph, fstart=t, fend=t+tspan)

In [ ]:
len(sorted(collider.subgraph.neartime(graph2, t, t+tspan).nodes()))

In [ ]:
look(experiment.graph, target=11212, jumps=10)